# Extract DEM from laz 
This notebook proccesses the raw USGS 3DEP LAZ file falling within a geometry provided into a single DEM.

**CONDA ENVIRONMENT**: `arcgispro-py3`

In [1]:
#Packages
from pathlib import Path
import pandas as pd
import arcpy

arcpy.env.overwriteOutput = True

* Get the tile features (found in `common\USGS_3DEP\USGS_Tiles\USGS3DEP_Indices.gdb`)

In [2]:
#Get the geodatabase containing USGS 3DEP Tiles
tile_fc_gdb = Path(r'\\ns-gis\DitchMapping\common\USGS_3DEP\USGS_Tiles\USGS3DEP_Indices.gdb')
if not tile_fc_gdb.exists():
    print(f'Failed to find Tile geodatabase: {tile_fc_gdb}')

#Get the feature class
tile_fc = str(tile_fc_gdb/'USGS_Tiles')
if not arcpy.Exists(tile_fc):
    print(f'Failed to find the USGS 3DEP Tile feature class: {tile_fc}')
else:
    print('Tiles Feature class found!')

Tiles Feature class found!


* Get the list of Duke LAS files used to define which USGS Tiles to get

In [3]:
#Get the Duke las files
the_folder = Path(r'\\ns-gis\DitchMapping\common\Duke\Hoffman')
las_files = list(the_folder.glob('*.las'))
las_files

[WindowsPath('//ns-gis/DitchMapping/common/Duke/Hoffman/Hof01_Final.las'),
 WindowsPath('//ns-gis/DitchMapping/common/Duke/Hoffman/Hof02_Final.las'),
 WindowsPath('//ns-gis/DitchMapping/common/Duke/Hoffman/Hof04_Final.las'),
 WindowsPath('//ns-gis/DitchMapping/common/Duke/Hoffman/Hof07_Final.las')]

* Process a single las file
    - Create a `lasd` file from the `las` file
    - Get the extent of the `lasd` file
    - Create a polygon from the extent
    - Create a feature class contaning the extent

In [4]:
#Get the footprint from a las file
las_file = las_files[3]
#Create a lasd file
lasd_file = str(las_file) + "d"
arcpy.management.CreateLasDataset(str(las_file),lasd_file)
#Get the extent
lasd_info = arcpy.da.Describe(lasd_file)
las_extent = lasd_info['extent']
xMin = las_extent.XMin
xMax = las_extent.XMax
yMin = las_extent.YMin
yMax = las_extent.YMax
sr = lasd_info['spatialReference']
#Create a polygon of the outline
array = arcpy.Array([
    arcpy.Point(xMin, yMin),
    arcpy.Point(xMin, yMax),
    arcpy.Point(xMax, yMax),
    arcpy.Point(xMax, yMin),
    arcpy.Point(xMin, yMin)  # Close the polygon
])

# Create polygon geometry
polygon = arcpy.Polygon(array, sr)  # Use appropriate EPSG code here
# Create the feature class from the polygon
footprint_fc = arcpy.management.CreateFeatureclass(
    out_path='memory',
    out_name='footprint',
    geometry_type="POLYGON",
    spatial_reference=polygon.spatialReference
)[0]

#Insert the footprint polygon
with arcpy.da.InsertCursor(footprint_fc, ["SHAPE@"]) as cursor:
    cursor.insertRow([polygon])

* Select all the USGS 3DEP Tiles that intersect the extent feature class
    - Create a feature layer of the tiles
    - Use `SelectLayerByLocation` to find intersecting 3DEP Tiles

In [5]:
#Create a feature layer of the Tiles
arcpy.management.MakeFeatureLayer(
    in_features = tile_fc,
    out_layer = "tiles_lyr"
)

#Find the tiles intersecting the footprint
arcpy.management.SelectLayerByLocation(
    in_layer="tiles_lyr",
    overlap_type="INTERSECT",
    select_features=footprint_fc,
    selection_type="NEW_SELECTION"
)

<Result 'tiles_lyr'>

* Create a dataframe from the selected tiles
    - Create columns containing the 3DEP Project and the tile name 

In [6]:
#Read the selected tiles into a dataframe
df = pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(
    in_table='tiles_lyr',
    field_names=['location']
))
#Split the location path into elements and keep the ones we want (project and tile name)
df = df['location'].str.split('/', expand=True)[[4,7]].rename(columns={4:'project',7:'tile'})
df

,project,tile
0,NC_HurricaneFlorence_3_2020,18STD27908685.las
1,NC_HurricaneFlorence_3_2020,18STD28058685.las
2,NC_HurricaneFlorence_3_2020,18STD27978692.las
3,NC_HurricaneFlorence_3_2020,18STD27908692.las
4,NC_HurricaneFlorence_3_2020,18STD28058692.las
5,NC_HurricaneFlorence_3_2020,18STD27978685.las


* Fetch the laz files, converting each to a DEM

In [7]:
#Create a scratch folder to hold intermediate files
scratch_fldr = Path.cwd().parent/'data'/'LAS_scratch'

#See if we can remove the folder (i.e. if it doesn't exist already)
if scratch_fldr.exists():
    delete_ok = False
    print(f'{scratch_fldr} already exists. Will not be deleted.')
else:
    delete_ok = True
    scratch_fldr.mkdir(parents=True,exist_ok=True)
    print(f'{scratch_fldr} created. Will be deleted on completion')

d:\jpfay\Fetch-USGS-3DEP\data\LAS_scratch already exists. Will not be deleted.


In [8]:
#Fetch the tiles from the Box drive
box_path = Path.home()/'Box'/ 'PROJECT 05-21-2025□ USGS3DEP'
if not box_path.exists(): print('Unable to connect to Box folder')
the_dems = []

for the_idx in df.index.values: 
    #Get the project folder
    the_folder = box_path / df.loc[the_idx,'project']
    if not the_folder.exists(): 
        print(f'unable to locate {the_folder}')
        break
    #Get the tile in the folder
    the_tile = the_folder / df.loc[the_idx,'tile'].replace('.las','.laz')
    if not the_tile.exists(): 
        print(f'unable to locate {the_tile}')
        break
    #Convert the laz file to a lasd file
    lasd_filename = str(the_tile.name).replace('.laz','.las')
    if (scratch_fldr/lasd_filename).exists():
        print(f'las file already created; skipping')
    else:
        print(f'Converting {the_tile.name} to {lasd_filename}')
        arcpy.conversion.ConvertLas(
            in_las = str(the_tile),
            target_folder = str(scratch_fldr),
            las_options=['REARRANGE_POINTS'],
            out_las_dataset=lasd_filename
        )
    #Filter out ground points (Class=2) and convert to raster
    arcpy.management.MakeLasDatasetLayer(
        in_las_dataset=str(scratch_fldr/lasd_filename),
        out_layer='ground_las',
        class_code=[2]
    )
    #Convert to raster
    las_dem = scratch_fldr/f'DEM_{lasd_filename[:-4]}.tif'
    if las_dem.exists():
        print(f'{las_dem} exists; skipping')
    else:
        print(f'Creating DEM {las_dem}')
        arcpy.conversion.LasDatasetToRaster(
            in_las_dataset='ground_las', 
            out_raster=str(las_dem), 
            value_field='ELEVATION',
            interpolation_type='BINNING AVERAGE LINEAR', 
            data_type='FLOAT', 
            sampling_type='CELLSIZE', 
            sampling_value=0.5, 
            z_factor=1
            )
    #Add the dem to the list of dems
    the_dems.append(las_dem)

Converting 18STD27908685.laz to 18STD27908685.las
Creating DEM d:\jpfay\Fetch-USGS-3DEP\data\LAS_scratch\DEM_18STD27908685.tif
Converting 18STD28058685.laz to 18STD28058685.las
Creating DEM d:\jpfay\Fetch-USGS-3DEP\data\LAS_scratch\DEM_18STD28058685.tif
Converting 18STD27978692.laz to 18STD27978692.las
Creating DEM d:\jpfay\Fetch-USGS-3DEP\data\LAS_scratch\DEM_18STD27978692.tif
Converting 18STD27908692.laz to 18STD27908692.las
Creating DEM d:\jpfay\Fetch-USGS-3DEP\data\LAS_scratch\DEM_18STD27908692.tif
las file already created; skipping
d:\jpfay\Fetch-USGS-3DEP\data\LAS_scratch\DEM_18STD28058692.tif exists; skipping
Converting 18STD27978685.laz to 18STD27978685.las
Creating DEM d:\jpfay\Fetch-USGS-3DEP\data\LAS_scratch\DEM_18STD27978685.tif


In [10]:
#Mosaic all the DEMs and clip them to the Duke LAS footprint
output_name = las_file.name[:-4]+'.tif'
print(f'Mosaic rasters to {scratch_fldr/output_name}')
arcpy.management.MosaicToNewRaster(
    input_rasters = [str(d) for d in the_dems],
    output_location= str(scratch_fldr),
    raster_dataset_name_with_extension=output_name,
    pixel_type='32_BIT_FLOAT',
    cellsize=0.5,
    number_of_bands=1,
    mosaic_method='BLEND',
    mosaic_colormap_mode='FIRST'
)

Mosaic rasters to d:\jpfay\Fetch-USGS-3DEP\data\LAS_scratch\Hof07_Final.tif


<Result 'd:\\jpfay\\Fetch-USGS-3DEP\\data\\LAS_scratch\\Hof07_Final.tif'>